![alt text](<https://www.et-gw.eu/images/et-new-logo.png>)

#  XIII ET Symposium Hakathon session


#### Tutorial on ``gwfast``

This notebook is a simple tutorial to start playing around with the ``gwfast`` software, which is suitable for SNR and Fisher-matrix based parameter estimation forecasts on large catalogues of events.



## Installation for Google Colab

In [ ]:
#! pip install -q 'git+https://github.com/CosmoStatGW/gwfast' 

**Note**: Using Google Colab, you need to restart the kernel runtime after installation.

## Now import some packages

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import h5py

import copy
from astropy.cosmology import Planck18

In [2]:
from gwfast import gwfastGlobals as glob

from gwfast.waveforms import IMRPhenomD_NRTidalv2
from gwfast.signal import GWSignal
from gwfast.network import DetNet
from gwfast.fisherTools import CovMatr, compute_localization_region, check_covariance, fixParams
from gwfast import gwfastUtils as utils

## COMPLETE EXAMPLE: GW170817 @ ET!

###  We use the Sardinia site for definiteness

In [ ]:
alldetectors = copy.deepcopy(glob.detectors)
print('All available detectors are: '+str(list(alldetectors.keys())))

# select only ET in Sardinia
ETSdet = {det:alldetectors[det] for det in ['ETS']}
print('Using detector '+str(list(ETSdet.keys())))


In [4]:
# We use the ET-D psds
ETSdet['ETS']['psd_path'] = os.path.join(glob.detPath, 'ET-0000A-18.txt')

### Initialise the signals and then the network 
(in this case we have a single triangular detector, but we will see later why this syntax is useful)

In [ ]:
mywf = IMRPhenomD_NRTidalv2(fRef=20.)

mySignals = {}

for d in ETSdet.keys():

    mySignals[d] = GWSignal(mywf, 
                psd_path=ETSdet[d]['psd_path'],
                detector_shape = ETSdet[d]['shape'],
                det_lat= ETSdet[d]['lat'],
                det_long=ETSdet[d]['long'],
                det_xax=ETSdet[d]['xax'], 
                verbose=False,
                useEarthMotion = True,
                fmin=2.) 
        
myNet = DetNet(mySignals, verbose=False)      

### Now we build a dictionary containing the parameters of GW170817

In [ ]:
# Median values of the posterior samples for all the parameters, 
# except psi and the coalescence phase that are set to 0

z = np.array([0.00980])
tGPS = np.array([1187008882.4])

GW170817 = {'Mc':np.array([1.1859])*(1.+z), 
            'dL':Planck18.luminosity_distance(z).value/1000., 
            'theta':np.array([np.pi/2. + 0.4080839999999999]), 
            'phi':np.array([3.4461599999999994]),
            'iota':np.array([2.545065595974997]), 
            'psi':np.array([0.]),
            'tcoal':utils.GPSt_to_LMST(tGPS, lat=0., long=0.), # GMST is LMST computed at long = 0° 
            'eta':np.array([0.24786618323504223]), 
            'Phicoal':np.array([0.]), 
            'chi1z':np.array([0.005136138323169717]), 
            'chi2z':np.array([0.003235146993487445]), 
            'Lambda1':np.array([368.17802383555687]), 
            'Lambda2':np.array([586.5487031450857])
           }

print('Parameters for GW170817 are:')
GW170817

### Let's see how the signal looks like at one of the interferometers

In [ ]:
# first define the frequency grid
fmax = mywf.fcut(**GW170817)-10 # We do not get to the cut this way
fgrid = np.geomspace(1.,fmax,10000)

AmplatET_p, AmplatET_c = myNet.signals['ETS'].GWAmplitudes(GW170817, fgrid, rot=120.)

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

ax.plot(fgrid, 2.*np.sqrt(fgrid)*np.sqrt(AmplatET_p**2 + AmplatET_c**2), linewidth=2., label='GW170817')
ax.plot(myNet.signals['ETS'].strainFreq, np.sqrt(myNet.signals['ETS'].noiseCurve), color='C2', linewidth=2., label='ET-D ASD')

ax.set_xlim(1.,fmax)
ax.set_ylim(1e-25, 1e-20)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('f [Hz]', fontsize=15)
ax.set_ylabel(r'$2 \ |\tilde{h}| \ \sqrt{f}\ [Hz^{-1/2}]$', fontsize=15)
plt.grid(linestyle='dotted', linewidth='0.6', which='both')
ax.legend(loc='upper right', fontsize=15, ncol=1, fancybox=True)

plt.show()

In [ ]:
# Compute the time to coalescence
ft = 2.
t_to_coal = mywf.tau_star(ft,**GW170817)
print('The time to coalescence at %d Hz is %.0f hours!'%(ft, t_to_coal/(3600.)))

### Compute the expected matched-filter SNR

In [ ]:
SNR = myNet.SNR(GW170817)
print('SNR for GW170817 at ET is %.2f'%SNR[0])

### Compute the total Fisher matrix

In [ ]:
totF = myNet.FisherMatr(GW170817)
print('The computed Fisher matrix has shape %s'%str(totF.shape))

In [ ]:
# Check e.g. that the (dL,dL) element corresponds to (SNR/dL)^2
ParNums = mywf.ParNums
dL_Num = ParNums['dL']
print('The relative difference is %.2e !'%((1 - totF[ParNums['dL'],ParNums['dL'],:]/(SNR/GW170817['dL'])**2)[0]))


### Compute the covariance and perform some checks

In [11]:
totCov, inversion_err = CovMatr(totF)


In [ ]:
_ = check_covariance(totF, totCov)


### Now try to eliminate the row corresponding to $\delta\tilde{\Lambda}$, and see that the inversion error lowers

In [ ]:
ParNums = mywf.ParNums

newFish, newPars = fixParams(totF, ParNums, ['deltaLambda'])

print('Now the Fisher matrix has shape %s'%str(newFish.shape))

newCov, new_inversion_err = CovMatr(newFish)

_ = check_covariance(newFish, newCov)


### Finally compute the localisation region

In [ ]:
skyArea = compute_localization_region(newCov, newPars, GW170817['theta'])
print('The estimated sky location is %.1f deg^2'%skyArea)
